# `dask.delayed`: processes vs threads

Here we use a (quite uneficient) python implementation of the euclidean distance matrix to understand how `dask.delayed` behaves with python code. Remember that before, what we run with `dask.delayed` was Scipy's `cdist` function.

In [1]:
import dask
import numpy as np

In [2]:
def euclidean_distance_matrix(x, y):
    num_samples = x.shape[0]
    dist_matrix = np.empty((num_samples, num_samples))
    for i, xi in enumerate(x):
        for j, yj in enumerate(y):
            diff = xi - yj
            dist_matrix[i][j] = diff.sum()
    return dist_matrix

In [3]:
x = np.random.random([1000, 50])

In [4]:
%%time
edm = euclidean_distance_matrix(x, x)

CPU times: user 3.16 s, sys: 5.65 ms, total: 3.17 s
Wall time: 3.16 s


<mark>Question</mark>: The following dask graph runs `euclidean_distance_matrix` twice using the same input data. From the time measured in the previous cell, estimate how long it will take to run the graph? Run the cells and check your answer.

In [5]:
graph = [dask.delayed(euclidean_distance_matrix)(x, x),
         dask.delayed(euclidean_distance_matrix)(x, x)]

In [8]:
%%time
edm = dask.compute(graph, scheduler='threads')
# not releasing the gil -> not really multithreaded

CPU times: user 6.58 s, sys: 23.9 ms, total: 6.61 s
Wall time: 6.59 s


<mark>Question</mark>: Estimate how long it will take to run the follwing cell. Run it and check your answer.

In [9]:
%%time
edm = dask.compute(graph, scheduler='processes')
# releaseing the gil

CPU times: user 51.8 ms, sys: 76.2 ms, total: 128 ms
Wall time: 4.01 s


<mark>Question</mark>: Could you explain the results?